In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import shutil
import tensorflow as tf
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

/Users/jegan/anaconda/lib/python3.6/site-packages/matplotlib/font_manager.py:280: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [2]:
cwd = os.getcwd()
print(cwd)

/Users/jegan/workspace/Google3.0


In [3]:
lnn =  pd.read_csv('final_input.csv')

lnn =  pd.read_csv('final_input.csv')
#rename UNNAMED:0 to INDEX
lnn.rename(columns={'Unnamed: 0': 'INDEX'}, inplace=True)

# drop UNNAMED:0.1
lnn = lnn.drop('INDEX', axis=1)

month_dict = {'JAN' : 1,
        'FEB' : 2,
        'MAR' : 3,
        'ARP' : 4, #whoops
        'MAY' : 5,
        'JUN' : 6,
        'JUL' : 7,
        'AUG' : 8,
        'SEP' : 9, 
        'OCT' : 10,
        'NOV' : 11,
        'DEC' : 12}

# convert months to numerical
from time import strptime
lnn['MONTH'] = lnn['MONTH'].apply(lambda x: month_dict[x])

lnn.head()

,STATE,YEAR,MONTH,TMP_HIGH,TMP_AVG,TMP_LOW,DP_HIGH,DP_AVG,DP_LOW,HUM_HIGH,...,VIS_LOW,WIND_HIGH,WIND_AVG,PRECIP,NUMDAY_RAIN,NUM_DAYS_SNOW,NUM_DAYS_FOG,NUM_DAYS_THNDRSTRM,PRICE,DISCOUNTED_PRICE
0,IL,1986,1,55.953658,27.617217,-2.980925,42.880246,19.992262,-8.511984,99.026800,...,0.186718,29.600197,11.077783,0.479003,2.544772,7.584738,9.707042,0.000000,2.35,2.349417
1,IL,1986,2,53.838886,28.731112,0.353638,46.537659,23.958440,-4.555208,99.510367,...,0.109089,24.679482,9.622850,1.722515,5.510271,9.231277,15.203328,0.658457,2.38,2.379410
2,IL,1986,3,77.668458,41.937001,13.926317,51.205991,29.862688,2.748564,99.138166,...,0.634101,36.374250,12.349509,15.116988,5.934169,3.868506,9.890731,1.055313,2.35,2.349417
3,IL,1986,4,85.651529,52.949283,25.351909,60.274139,37.049373,11.479874,99.558871,...,0.474198,28.918771,10.770362,17.947352,9.195066,1.960945,7.611098,2.058521,2.34,2.339420
4,IL,1986,5,85.415668,62.338688,36.602102,61.467698,47.555928,27.795232,95.246992,...,0.280723,33.072546,8.983943,15.212715,9.610446,0.327268,14.161837,4.663441,2.45,2.449393


In [111]:
x_train, x_test, y_train, y_test = train_test_split(lnn.iloc[:,2:25], lnn.iloc[:,26], test_size=0.2)

In [5]:
trainsize = int(len(y_train) * 0.8)
avg = np.mean(y_train[:trainsize])
rmse = np.sqrt(np.mean((y_train[trainsize:] - avg)**2))
print('Just using average={0} has RMSE of {1}'.format(avg, rmse))

Just using average=2.990190636112074 has RMSE of 1.2313843379890743


In [36]:
trainsize = int(len(y_train) * 0.8)
testsize = len(y_train) - trainsize
npredictors = len(x_train.columns)
noutputs = 1
shutil.rmtree('./trained_model_linear', ignore_errors=True) # so that we don't load weights from previous runs
estimator = tf.contrib.learn.LinearRegressor(model_dir='./trained_model_linear',
                                             feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(x_train.values))

print("starting to train ... this will take a while ...")
def input_fn(features, y_train):
  return tf.constant(features.values), tf.constant(y_train.values.reshape(len(y_train), noutputs))
estimator.fit(input_fn=lambda: input_fn(x_train[:trainsize], y_train[:trainsize]), steps=100000)

pred = list(estimator.predict(x_train[trainsize:].values))
rmse = np.sqrt(np.mean(np.power((y_train[trainsize:].values - pred), 2)))
print('LinearRegression has RMSE of {0}'.format(rmse))

starting to train ... this will take a while ...
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
LinearRegression has RMSE of 1.0873111383816239


In [40]:
r2_score(y_train[trainsize:].values,pred)

0.21998455257571436

In [41]:
trainsize = int(len(y_train) * 0.8)
testsize = len(y_train) - trainsize
npredictors = len(x_train.columns)
noutputs = 1
tf.logging.set_verbosity(tf.logging.WARN) # change to INFO to get output every 100 steps ...
shutil.rmtree('./trained_model', ignore_errors=True) # so that we don't load weights from previous runs
estimator = tf.contrib.learn.DNNRegressor(model_dir='./trained_model',
                                          hidden_units=[3, 3],                             
                                          feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(x_train.values))

print("starting to train ... this will take a while ...")
def input_fn(features, y_train):
  return tf.constant(features.values), tf.constant(y_train.values.reshape(len(y_train), noutputs))
estimator.fit(input_fn=lambda: input_fn(x_train[:trainsize], y_train[:trainsize]), steps=100000)

pred = list(estimator.predict(x_train[trainsize:].values))
rmse = np.sqrt(np.mean(np.power((y_train[trainsize:].values - pred), 2)))
print('Neural Network Regression has RMSE of {0}'.format(rmse))

starting to train ... this will take a while ...
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Neural Network Regression has RMSE of 1.2308321597202898


In [42]:
r2_score(y_train[trainsize:].values,pred)

0.00047606134964406355

In [55]:
trainsize = int(len(y_train) * 0.8)
testsize = len(y_train) - trainsize
npredictors = len(x_train.columns)
noutputs = 1
shutil.rmtree('./trained_model_comb', ignore_errors=True) # so that we don't load weights from previous runs
estimator = tf.contrib.learn.DNNLinearCombinedRegressor(model_dir='./trained_model_comb',                           
                                          linear_feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(x_train.values),
                                          dnn_hidden_units = [3,3],
                                          dnn_feature_columns=tf.contrib.learn.infer_real_valued_columns_from_input(x_train.values))

print("starting to train ... this will take a while ...")
def input_fn(features, y_train):
  return tf.constant(features.values), tf.constant(y_train.values.reshape(len(y_train), noutputs))
estimator.fit(input_fn=lambda: input_fn(x_train[:trainsize], y_train[:trainsize]), steps=1000000)

pred = list(estimator.predict(x_train[trainsize:].values))
rmse = np.sqrt(np.mean(np.power((y_train[trainsize:].values - pred), 2)))
print('DNN+Linear has RMSE of {0}'.format(rmse))

Instructions for updating:
Please set fix_global_step_increment_bug=True and update training steps in your pipeline. See pydoc for details.
starting to train ... this will take a while ...
Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
Instructions for updating:
Please switch to predict_scores, or set `outputs` argument.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
DNN+Linear has RMSE of 1.0860236020072753


In [57]:
r2_score(y_train[trainsize:].values,pred)

0.22183076495637633

In [112]:
from tensorflow.contrib.tensor_forest.python import tensor_forest
from tensorflow.contrib.learn.python import SKCompat
from tensorflow.python.estimator.inputs import numpy_io


trainsize = int(len(y_train) * 0.8)
testsize = len(y_train) - trainsize
npredictors = len(x_train.columns)
noutputs = 1
shutil.rmtree('./trained_model_rf', ignore_errors=True) # so that we don't load weights from previous runs
hparams = tensor_forest.ForestHParams(
        num_trees=3,
        max_nodes=1000,
        num_classes=3,
        num_features=4,
        split_after_samples=20,
        regression=True)
estimator = tf.contrib.tensor_forest.client.random_forest.TensorForestEstimator(hparams.fill(), model_dir='./trained_model_rf')


# def input_fn(features, y_train):
#   return tf.constant(features.values), tf.constant(y_train.values.reshape(len(y_train), noutputs))

# estimator.fit(input_fn=lambda: input_fn(x_train[:trainsize], y_train[:trainsize]), steps=100000)

#print(x_train.shape)
#print(pd.DataFrame({"x": x_train}, index=list(range(0,23))))
#print(pd.Series(y_train))




train_input_fn = tf.estimator.inputs.pandas_input_fn(
      x=x_train,
      y=y_train,
      batch_size=10,
      num_epochs=None,
      shuffle=True)

# train_input_fn = numpy_io.numpy_input_fn(
#       x={'x': np.array(x_train).astype(np.int32)},
#       y=np.array(y_train).astype(np.int32),
#       batch_size=10,
#       num_epochs=None,
#       shuffle=True)


estimator.fit(input_fn=train_input_fn, steps=1000)

#estimator.fit(x=x_train, y=y_train, batch_size=10, steps=10000)




#from tensorflow.contrib.learn.python import SKCompat

#regressor = SKCompat(estimator)

#x_val = [tf.cast(x_col, tf.float32) for x_col in x_train]
#x_val = x_train.astype(tf.float32)
#regressor.fit(x=tf.cast(x_train,tf.float32), y=y_train, batch_size=10, steps=10000)



TypeError: Tensors in list passed to 'values' of 'ConcatV2' Op have types [float64, float64, float64, float64, float64, float64, float32, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64, float64] that don't all match.